In [1]:
data_var = '2024-01-30'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
11372,2024-01-30,França Pro B,16:00,Boulazac,Rouen,1.55,2.34,161.5,1.87,1.79,-4.5,1.98,1.74,dlLlVenL,0.645161,0.427350,0.534759,0.558659,0.072512,144.376,57.020802,0.394947,1.8,1.643168,0.912871,90.06,1.696,0.675892,0.398521,23.0,138.692,36.524024,0.263346,1.8,1.643168,0.912871,113.40,1.642,0.230478,0.140364,37.0,79,90,1.14,1.26,109.714,161.390,0.287205,0.030912,0.091240,1.69,0.338,1.627219,0.000000,0.0,0.000000,-0.61,-0.122,-10.983607,0.000000,0.0,0.000000
11373,2024-01-30,França Pro B,16:00,Chalons-Reims,Nantes,1.43,2.68,158.5,1.87,1.79,-6.5,2.08,1.67,EFSsCGuq,0.699301,0.373134,0.534759,0.558659,0.072435,133.702,39.626749,0.296381,1.8,1.643168,0.912871,183.20,1.682,0.374793,0.222826,26.0,158.682,43.792051,0.275974,1.8,1.643168,0.912871,119.60,2.034,0.688716,0.338602,-3.0,80,92,2.29,1.30,110.256,188.822,0.430114,0.030912,0.154621,-2.26,-0.452,-0.951327,0.000000,0.0,0.000000,-0.46,-0.092,-18.260870,0.000000,0.0,0.000000
11374,2024-01-30,França Pro B,16:00,Denain-Voltaire,Antibes,1.79,1.93,159.5,1.87,1.79,-2.5,2.09,1.66,fPzjAfAe,0.558659,0.518135,0.534759,0.558659,0.076794,218.128,106.938147,0.490254,1.2,1.643168,1.369306,339.34,2.854,1.325002,0.464261,-27.0,147.228,33.454018,0.227226,2.4,1.341641,0.559017,153.00,1.760,0.394905,0.224378,30.0,94,90,3.61,1.70,118.006,198.732,0.053223,0.030912,0.162163,-1.93,-0.386,-2.046632,0.000000,0.0,0.000000,0.23,0.046,20.217391,0.000000,0.0,0.000000
11375,2024-01-30,França Pro B,16:00,La Rochelle,Evreux,1.19,4.37,156.5,1.83,1.87,-11.5,1.97,1.75,MVSyYc22,0.840336,0.228833,0.546448,0.534759,0.069169,124.266,29.316515,0.235917,2.4,1.341641,0.559017,156.88,1.604,0.454896,0.283601,4.0,153.982,69.110255,0.448820,1.2,1.643168,1.369306,102.81,2.012,0.810352,0.402759,-14.0,74,69,2.12,1.49,100.254,232.566,0.808849,0.015289,0.083636,1.16,0.232,0.818966,0.000000,0.0,0.000000,1.27,0.254,13.267717,0.000000,0.0,0.000000
11376,2024-01-30,Irã Superliga,09:30,Mahram Tehran,Mes Kerman,1.24,3.77,161.5,1.81,1.89,-10.5,2.03,1.67,Ay3kDJ1I,0.806452,0.265252,0.552486,0.529101,0.071704,131.124,31.519671,0.240381,1.8,1.643168,0.912871,150.52,1.460,0.378220,0.259054,46.0,378.596,494.427729,1.305951,1.2,1.643168,1.369306,184.96,5.110,6.690262,1.309249,-18.0,106,68,1.42,2.72,139.708,486.756,0.714164,0.030578,0.137599,-0.77,-0.154,-1.558442,0.000000,0.0,0.000000,8.47,1.694,1.635183,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11472,2024-01-30,Eua Ncaa,21:00,Ohio State,Illinois,2.18,1.69,149.5,1.91,1.80,10.5,1.25,0.00,0SbLUUd6,0.458716,0.591716,0.523560,0.555556,0.050432,120.400,20.665797,0.171643,0.6,1.341641,2.236068,126.44,1.848,0.396951,0.214800,-46.0,109.898,35.641386,0.324313,1.8,1.643168,0.912871,75.60,1.346,0.292455,0.217277,32.0,58,70,2.18,1.08,92.462,200.836,0.179061,0.041931,1.414214,-0.52,-0.104,-11.346154,0.831314,0.9,0.068686,-3.27,-0.654,-1.055046,0.000000,0.0,0.000000
11473,2024-01-30,Eua Ncaa,21:00,Georgia Tech,North Carolina,4.89,1.18,152.5,1.83,1.83,10.5,1.91,1.80,0U1amHCG,0.204499,0.847458,0.546448,0.546448,0.051957,390.372,308.714683,0.790822,0.6,1.341641,2.236068,249.24

## Apostas do dia

### Back Home V3 (QCut)

In [8]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
11404,21:00,Eua Ncaa,Central Michigan,Northern Illinois,1.58,Back Home
11406,21:30,Eua Nba,New York Knicks,Utah Jazz,1.61,Back Home
11419,14:00,Israel Liga Leumit,Ramat Hasharon,Ironi Nahariya,2.45,Back Home
11427,07:00,Coreia Do Sul Kbl,Goyang Sono,LG Sakers,2.65,Back Home
11436,16:00,Europa Eurocopa,Venezia,Prometey,1.90,Back Home
11445,22:00,Eua Nba,Chicago Bulls,Toronto Raptors,1.74,Back Home
11450,21:00,Eua Ncaa,Ball State,Bowling Green,1.90,Back Home
11458,14:00,Europa Liga Dos Campeões,Hapoel Holon,AEK Athens,2.08,Back Home
11465,23:00,Eua Ncaa,Air Force,Wyoming,1.55,Back Home
